# Setting up experiment parameters

In [ ]:
EPOCHES = 2 # Experiments were set for 30 or 10 epoches

# Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.models import resnet18

import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import time
import random


%matplotlib inline

# Model classes

In [ ]:
class BaseNet(nn.Module):
    '''
    Base regression model
    '''
    def __init__(self, ini_bias=None):
        super().__init__()
        self.layers = nn.Sequential(
           nn.Linear(90, 120),
           nn.BatchNorm1d(120),
           nn.Dropout(0.2),
           nn.ReLU(),
           nn.Linear(120, 20),
           nn.BatchNorm1d(20),
           nn.Dropout(0.2),
           nn.ReLU(),
           nn.Linear(20, 1)
        )
        
    def forward(self, x):
        return self.layers(x)

In [ ]:
class StarRegression(nn.Module):
    def __init__(self, trained_list, BaseClass, warmup=None, *args, **kwargs):
        '''
        Constructs Star aggregation of models
        uses segments from empirical minimizers (trained models) via sigmoid

        args:
            trained_list - list of trained models (will be copied)
            BaseClass - class of new model,
                        which will be connected with trained_model
                        and trained alongside weights of models (self.l)
            *args, **kwargs - arguments for BaseClass initialization 
        '''

        super().__init__()

        self.trained_models = nn.ModuleList()
        for model in trained_list:
            self.trained_models.append(copy.deepcopy(model))
            for p in self.trained_models[-1].parameters():
                p.requires_grad = False
        if warmup is not None:
            self.training_model = copy.deepcopy(warmup)
            for p in self.training_model.parameters():
                p.requires_grad = True
        else:
            self.training_model = BaseClass(*args, **kwargs)

        self.weights = nn.Parameter(torch.zeros(len(trained_list)+1))
        self.weights.requires_grad = True


    def forward(self, x):
        outs = []
        for model in self.trained_models:
            outs.append(model(x))
        outs.append(self.training_model(x))
        return torch.stack(outs, -1) @ nn.functional.softmax(self.weights, dim=0)

In [ ]:
class ParallelRegression(nn.Module):
    def __init__(self, BaseClass, model_num=2, *args, **kwargs):
        '''
        args:
            BaseClass - class of models,
                        which will be connected similarly to SegmentStarRegression
                        but trained in parallel
            model_num - number of base models
            *args, **kwargs - arguments for BaseClass initialization
        '''
        super().__init__()

        self.models = nn.ModuleList()
        for _ in range(model_num):
            self.models.append(BaseClass(*args, **kwargs))
        self.linear = nn.Linear(model_num, 1, bias=True)


    def forward(self, x):
        outs = []
        for model in self.models:
            outs.append(model(x))
        return self.linear(torch.stack(outs, -1))

# Preparing dataset

In [ ]:
df = pd.read_csv('data/YearPredictionMSD.txt', header=None)

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

SEED = 0

set_random_seed(SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 128
MAX_D = 5
NUM_EXPERIMENTS = 5


features_scaler = StandardScaler()
scaled_X_train = features_scaler.fit_transform(X_train)
scaled_X_test  = features_scaler.transform(X_test)

target_scaler = StandardScaler()
scaled_y_train = target_scaler.fit_transform(y_train.reshape(-1, 1)).squeeze()
scaled_y_test = target_scaler.transform(y_test.reshape(-1, 1)).squeeze()


train_set = TensorDataset(torch.Tensor(scaled_X_train), torch.Tensor(scaled_y_train))
train_dataloader = DataLoader(train_set, batch_size = batch_size, pin_memory=True, shuffle=True)

test_set = TensorDataset(torch.Tensor(scaled_X_test), torch.Tensor(scaled_y_test))
test_dataloader = DataLoader(test_set, batch_size = batch_size, pin_memory=True)

In [ ]:
Y_SCALE = target_scaler.scale_[0]
Y_SCALE

# Training functions

In [ ]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device='cpu'):
    '''
    Trains model for a single epoch
    
    Args:
        model - model to be trained
        train_dataloader - dataloader of training dataset
        criterion - loss criterion
        optimizer - optimizer
        device - device to compute on
    
    returns:
        average loss on training dataset
    '''
    model.train()

    loss_sum = 0
    acc_sum = 0
    total = 0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x).squeeze()
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_sum += loss.item() * y.shape[0]
        total += y.shape[0]

    return loss_sum / total


def train(model, train_dataloader, epoch_num, optimizer, scheduler=None,
          device='cpu', plot_freq=2, y_scale=Y_SCALE, name=''):
    '''
    Trains model for given number of epoches
    
    Args:
        model - model to be trained
        train_dataloader - dataloader of training dataset
        epoch_num - number of epoches to train
        optimizer - optimizer
        scheduler - optimizer scheduler
        device - device to compute on
        plot_freq - if int, graph is plotted every plot_freq epoches, else only loss is printed
        y_scale - scale of targets
        name - name of model
    
    returns:
        losses - list of loss on training dataset
        training time - total time spent on training
    '''

    criterion = nn.MSELoss()
    start_time = time.time()

    train_losses = []
    for epoch in range(epoch_num):
        loss = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
        train_losses += [loss * y_scale**2]
        
        if scheduler is not None:
            scheduler.step()
            
        if isinstance(plot_freq, int) and epoch % plot_freq == plot_freq-1:
            fig, axs = plt.subplots(figsize=(8, 8))
            axs.plot(range(len(train_losses)), train_losses)
            axs.set(ylabel = 'MSE', xlabel='Epoch', title='Training Loss')
            axs.legend()
            
            clear_output()
            plt.show()
            print(name, 'epoch number ', epoch, '\n')
            print('time training:', round(time.time() - start_time, 1), 'sec')
            print('train loss:', np.sqrt(train_losses[-1]), '\n\n')

    training_time = round(time.time() - start_time, 1)

    return train_losses, training_time

In [ ]:
def get_mse_mae(model_list, dataloader, y_scale=Y_SCALE, device='cpu'):
    '''
    Evaluates model on all objects in dataset
    
    Args:
        model_list - model to be evaluated,
                     if list of models - ensemble of these models will be evaluated
        dataloader - dataloader of a dataset
        y_scale - scale of targets
        device - device to compute on
    
    returns:
        mse_sum, mae_sum - average MSE and MAE on given dataset (scaled by y_scale)
    '''
    if not isinstance(model_list, list):
        model_list = [model_list]
    for model in model_list:
        model.eval()

    mse_sum = 0
    mae_sum = 0
    total = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model_list[0](x).squeeze()
            for model in model_list[1:]:
                y_pred += model(x).squeeze()
            y_pred /= len(model_list)

            mse_sum += torch.sum((y_pred - y)**2).item()
            mae_sum += torch.sum(torch.abs(y_pred - y)).item()
            total += y.shape[0]
    
    mse_sum *= y_scale**2 / total
    mae_sum *= y_scale / total

    return mse_sum, mae_sum

In [ ]:
def snap_train_one_epoch(model, train_dataloader, criterion, optimizer, scheduler=None, device='cpu'):
    '''
    Trains model for a single epoch via snapshot technique
    
    Args:
        model - model to be trained
        train_dataloader - dataloader of training dataset
        criterion - loss criterion
        optimizer - optimizer
        device - device to compute on
    
    returns:
        average loss on training dataset
    '''
    model.train()

    loss_sum = 0
    total = 0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x).squeeze()
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_sum += loss.item() * y.shape[0]
        total += y.shape[0]
        if scheduler is not None:
            scheduler.step()
            
    return loss_sum / total


def snapshot_train(model, train_dataloader, epoch_num, device='cpu',
                   snap_freq=EPOCHES, plot_freq=2, scale=Y_SCALE, name=None):
    ''' 
    Trains model for given number of epoches via snapshot technique
    
    args:
        model - model to be trained
        train_dataloader - dataloader of training dataset
        epoch_num - amount of epoches to train
        device - device to compute on
        snap_freq - length of iteration of snapshot cycle (in epoches)
        plot_freq - if int, graph is plotted every plot_freq epoches, else only loss is printed
        y_scale - scale of targets
        name - name of model
    
    returns:
        snapshots - list containing copies of snapshots
        losses - list of losses on training dataset
        training time - list of times spent on training for each snapshot
    '''

    start_time = time.time()

    train_losses = []
    
    snapshots = []
    snap_times = []
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader) * snap_freq)

    for epoch in range(epoch_num):
        loss = snap_train_one_epoch(model, train_dataloader, criterion, optimizer, scheduler, device)
        train_losses += [loss * scale**2]
            
        if epoch % snap_freq == snap_freq - 1:
            snapshots += [copy.deepcopy(model)]
            snap_times += [round(time.time() - start_time)]

            optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader) * snap_freq)


        if isinstance(plot_freq, int) and epoch % plot_freq == plot_freq - 1:
            fig, axs = plt.subplots(figsize=(8, 8))
            axs.plot(range(len(train_losses)), train_losses)
            axs.set(ylabel = 'MSE', xlabel='Epoch', title='Training Loss')
            axs.legend()
            
            clear_output()
            plt.show()
            print(name, 'epoch number ', epoch, '\n')
            print('time training:', round(time.time() - start_time, 1), 'sec')
            print('train loss:', np.sqrt(train_losses[-1]), '\n\n')

    return snapshots, train_losses, np.array(snap_times)

In [ ]:
def add_res(results, name, add_row):
    '''
    Adds new row with index "name" to DataFrame
    Args:
        results -  pd.DataFrame containing results for all models
        name - name of new row
        add_row - row which will be added
    '''
    if name not in results.index:
        results.loc[name] = add_row
        return

    res_row = []
    for i, col in enumerate(results):
        res_row += [results.loc[name,col] + add_row[i]]
    results.loc[name] = res_row


def make_experiment(results, model_class=BaseNet, epoches=EPOCHES, base_am=MAX_D, warmup_epoches=EPOCHES//10):
    '''
    Run an experiment for non-snapshot classes
    Args:
        results - pd.DataFrame containing results for all models
        model_class - Class for each base block
        epoches - total number of epoches for each step of algorithm
        base_am - maximum d (there will be total of base_am + 1 blocks)
        warm_up_epoches - number of epoches for warm-up
    returns:
        results - changed pd.DataFrame containing old and new results
    '''

    bases = []
    
    criterion = nn.MSELoss()
    ensemble_times = []
    scheduler = None
    
    for i in range(base_am+1):
        bases.append(model_class())
        bases[i].to(device)

        optimizer = torch.optim.Adam(bases[i].parameters(), lr=1e-3, weight_decay=1e-3)
        losses, time = train(bases[i], train_dataloader, epoches,
                             optimizer, device=device, name="base model " + str(i+1))
    
        if len(ensemble_times):
            ensemble_times += [ensemble_times[-1] + time]
        else:
            ensemble_times += [time]

        mse, mae = get_mse_mae(bases[i], test_dataloader, Y_SCALE, device)
        
        add_res(results, 'Big NN 0', [[mse], [mae], [losses[-1]], [time]])
    
    # ensemble -----------------------------------------------------------------

    for i in range(base_am):
        mse, mae = get_mse_mae(bases[:i+2], test_dataloader, Y_SCALE, device)
        train_mse, train_mae = get_mse_mae(bases[:i+2], train_dataloader, Y_SCALE, device)
            
        add_res(results, 'Ensemble ' + str(i+1), [[mse], [mae], [train_mse], [ensemble_times[i+1]]])

    # star ---------------------------------------------------------------------

    star_models = []

    for i in range(base_am):
        name = 'Classic Star (no warm-up) ' + str(i+1)
        star = StarRegression(bases[:i+1], model_class).to(device)
        optimizer = torch.optim.Adam(star.parameters(), lr=1e-3, weight_decay=1e-3)
        
        losses, time = train(star, train_dataloader, epoches, optimizer, device=device, name=name)

        mse, mae = get_mse_mae(star, test_dataloader, Y_SCALE, device)
            
        add_res(results, name, [[mse], [mae], [losses[-1]], [time + ensemble_times[i]]])
  

    # warmup star ---------------------------------------------------------------------

    warmed = model_class().to(device)

    optimizer = torch.optim.Adam(warmed.parameters(), lr=1e-3, weight_decay=1e-3)
    _, warmup_time = train(warmed, train_dataloader, warmup_epoches,
                           optimizer, device=device, name='warmup')

    for i in range(base_am):
        name = 'Classic Star (new warm-up) ' + str(i+1)
        star = StarRegression(bases[:i+1], model_class, warmup=warmed).to(device)
        optimizer = torch.optim.Adam(star.parameters(), lr=1e-3, weight_decay=1e-3)
        
        losses, time = train(star, train_dataloader, epoches - warmup_epoches, optimizer, device=device, name=name)

        mse, mae = get_mse_mae(star, test_dataloader, Y_SCALE, device)
            
        add_res(results, name, [[mse], [mae], [losses[-1]], [time + warmup_time + ensemble_times[i]]])


    # parallel -----------------------------------------------------------------

    parallel_models = []

    for i in range(base_am):
        name = 'Big NN ' + str(i+1)
        parallel_model = ParallelRegression(model_class, i+2).to(device)

        optimizer = torch.optim.Adam(parallel_model.parameters(), lr=1e-3, weight_decay=1e-3)
        
        losses, time = train(parallel_model, train_dataloader, epoches, optimizer, device=device, name=name)

        mse, mae = get_mse_mae(parallel_model, test_dataloader, Y_SCALE, device)
        
        add_res(results, name, [[mse], [mae], [losses[-1]], [time]])

    return results

In [ ]:
def make_snapshot_experiment(results, model_class=BaseNet, epoches=EPOCHES, base_am=MAX_D, warmup_epoches=EPOCHES//10):
    '''
    Run an experiment for snapshot classes
    Args:
        results - pd.DataFrame containing results for all models
        model_class - Class for each base block
        epoches - total number of epoches for each step of algorithm
        base_am - maximum d (there will be total of base_am + 1 blocks)
        warm_up_epoches - number of epoches for warm-up
    returns:
        results - changed pd.DataFrame containing old and new results
    '''

    base = model_class().to(device)
    snapshots, losses, cum_times = snapshot_train(base, train_dataloader, epoches*(base_am+1),
                                                  device=device, snap_freq=epoches, name='snapshot')
    
    snap_times = cum_times.copy()
    snap_times[1:] = np.diff(cum_times)
        
    for i in range(len(snapshots)):
        snapshots[i].to(device)
        mse, mae = get_mse_mae(snapshots[i], test_dataloader, Y_SCALE, device)
        add_res(results, 'Snap Ensemble 0', [[mse], [mae], [losses[-1]], [snap_times[i]]])    
    

    # ensemble -----------------------------------------------------------------
    
    for i in range(base_am):
        mse, mae = get_mse_mae(snapshots[:i+2], test_dataloader, Y_SCALE, device)
        train_mse, train_mae = get_mse_mae(snapshots[:i+2], train_dataloader, Y_SCALE, device)
            
        add_res(results, 'Snap Ensemble ' + str(i+1), [[mse], [mae], [train_mse], [cum_times[i+1]]])

    # star ---------------------------------------------------------------------

    star_models = []

    for i in range(base_am):
        name = 'Snap Star (no warm-up) ' + str(i+1)
        star = StarRegression(snapshots[:i+1], model_class).to(device)
        optimizer = torch.optim.Adam(star.parameters(), lr=1e-3, weight_decay=1e-3)
        
        losses, time = train(star, train_dataloader, epoches, optimizer,device=device, name=name)

        mse, mae = get_mse_mae(star, test_dataloader, Y_SCALE, device)
            
        add_res(results, name, [[mse], [mae], [losses[-1]], [time + cum_times[i]]])
  

    # warmup star ---------------------------------------------------------------------

    warmed = model_class().to(device)

    optimizer = torch.optim.Adam(warmed.parameters(), lr=1e-3, weight_decay=1e-4)
    _, warmup_time = train(warmed, train_dataloader, warmup_epoches, optimizer, device=device, name='warmup')

    for i in range(base_am):
        name = 'Snap Star (new warm-up) ' + str(i+1)
        star = StarRegression(snapshots[:i+1], model_class, warmup=warmed).to(device)
        optimizer = torch.optim.Adam(star.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler=None
        
        losses, time = train(star, train_dataloader, epoches - warmup_epoches,
                optimizer, device=device, name=name)

        mse, mae = get_mse_mae(star, test_dataloader, Y_SCALE, device)
            
        add_res(results, name, [[mse], [mae], [losses[-1]], [time + warmup_time + cum_times[i]]])


    # snap warmup star ---------------------------------------------------------------------

    warmed = model_class().to(device)

    for i in range(base_am):
        name = 'Snap Star (shot warm-up) ' + str(i+1)
        star = StarRegression(snapshots[:i+1], model_class, warmup=snapshots[i]).to(device)
        optimizer = torch.optim.Adam(star.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler=None
        
        losses, time = train(star, train_dataloader, epoches, optimizer, device=device, name=name)

        mse, mae = get_mse_mae(star, test_dataloader, Y_SCALE, device)
            
        add_res(results, name, [[mse], [mae], [losses[-1]], [time + cum_times[i]]])

    return results

In [ ]:
set_random_seed(SEED)
results = pd.DataFrame({'MSE' : [], 'MAE' : [], 'Train MSE' : [], 'Time' : []})
for i in range(NUM_EXPERIMENTS):
    results = make_experiment(results,  warmup_epoches=1)

In [ ]:
set_random_seed(SEED)
for i in range(NUM_EXPERIMENTS):
    results = make_snapshot_experiment(results, warmup_epoches=1)

In [ ]:
results.to_csv(f'MSD {EPOCHES} epoch raw results.csv', index=False)

In [ ]:
results

In [ ]:
VAR = (np.std(scaled_y_train)*Y_SCALE)**2
print('Sample variance (for R2 score):', VAR)

In [ ]:
results = results.drop('Snap Star (no warm-up)1')

# Creating result table

In [ ]:
for idx in results.index:
    d = idx.rsplit(' ', 1)[1]
    results.loc[idx, 'd'] = d 

results.sort_values(by='d', inplace=True, ascending=False)

converted_list = []

for idx in results.index:
    name = idx.rsplit(' ', 1)[0]
    mean_mse = np.mean(results.loc[idx, 'MSE'])
    std_mse = np.std(results.loc[idx, 'MSE'])
    train_mse = np.mean(results.loc[idx, 'Train MSE'])
    mae = np.mean(results.loc[idx, 'MAE'])
    time = np.mean(results.loc[idx, 'Time'])
    d = int(results.loc[idx, 'd'])

    
    converted_list.append([name, d, str(np.round(mean_mse, 2)) + ' ± ' + str(np.round(std_mse, 2)),
                       np.round(mae, 2), np.round(1 - (mean_mse / VAR), 3), np.round(train_mse, 2), round(time)])


converted_results = pd.DataFrame(converted_list, 
        columns=['model', 'd', 'MSE', 'MAE', 'R2', 'TRAIN MSE', 'TIME (sec)'])

converted_results.to_csv(f'MSD {EPOCHES} epoch results.csv', index=False)

In [ ]:
converted_results